# Simple NNs with custom embeddings

## Importing data

In [1]:
import copy
from itertools import product
from numpy import isnan
import pandas as pd
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext.data as data
import sys

sys.path.append('../data_pipeline/')
import preprocessing as pre
from training import TrainingModule

In [2]:
train_data, test_data, val_data, TEXT, LABEL = pre.get_data(
    'train_small.csv', 'val_small.csv', 'test_small.csv', 50)

Connected!


../data_pipeline/preprocessing.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sans_nulls_df['alj_text'] = sans_nulls_df['alj_text'].str.slice(0, 1000000)


In [3]:
USE_CUDA = torch.cuda.is_available()

TEXT.build_vocab(train_data)
LABEL.build_vocab(train_data)

BATCH_SIZE = 5

train_it, test_it, val_it = data.BucketIterator.splits(
    (train_data, test_data, val_data), 
    batch_size = BATCH_SIZE,
    sort_key=lambda x: len(x.alj_text),
    sort_within_batch=True,
    device = torch.device('cuda' if USE_CUDA else 'cpu'))

## Simple NN Model

In [12]:
class WordEmbAvg(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, pad_idx, two_layers=True, dropout_p=0.0):
        
        super().__init__()
        
        # Define an embedding layer, a couple of linear layers, and 
        # the ReLU non-linearity.

        ##YOUR CODE HERE##
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        if two_layers == True:
            self.linear1 = nn.Linear(embedding_dim, hidden_dim)
            self.linear2 = nn.Linear(hidden_dim, output_dim) 
        else:
            self.linear1 = nn.Linear(embedding_dim, output_dim)
            self.linear2 = None
        self.relu = nn.ReLU()
        self.drop_layer = nn.Dropout(p=dropout_p)

        
        
    def forward(self, text):

        ##YOUR CODE HERE##
        embedded = self.embedding(text)
        embedded = embedded.mean(0)
        if not self.linear2:
            linear1_output = self.linear1(embedded)
            output = self.relu(linear1_output)
            output = self.drop_layer(output)
            return output
        else:
            linear1_output = self.linear1(embedded)
            linear2_input = self.relu(linear1_output)
            output = self.linear2(linear2_input)
            output = self.drop_layer(output)
            return output

## Training models

In [13]:
# Store training results

df = pd.DataFrame(columns=['architecture', 'embeddings',
                           'hidden', 'dropouts',
                           'learning_rate', 'epochs',
                           'dev_acc', 'dev_prec', 'dev_recall',
                           'metric'])

# Model architecture parameters
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_SIZES = [32, 64, 128, 256]
HIDDEN_SIZES = [10, 25, 40, 50]
OUTPUT_SIZE = 1
DROPOUTS = [0, 0.1, 0.25, 0.5, 0.75]
PADDING_IDX = TEXT.vocab.stoi[TEXT.pad_token]

# Model training hyperparameters
LEARNING_RATE = [0.01, 0.001, 0.0001]
train_len = 0
train_pos = 0
for batch in train_it:
    train_len += len(batch.decision_binary)
    train_pos += batch.decision_binary.sum().item()
POS_WEIGHT = torch.tensor([(train_len - train_pos) / train_pos])
if USE_CUDA:
    POS_WEIGHT = POS_WEIGHT.cuda()
EPOCHS = 10

# Iterator over various model parameters
param_iter = product (EMBEDDING_SIZES, HIDDEN_SIZES,
                      DROPOUTS, LEARNING_RATE)

# Magic loop
best_acc = (None, None)
best_rec = (None, None)
best_prec = (None, None)
for i, (embed_size, hidden_size, dropout, lr) in enumerate(param_iter):
    print(f'Architecture #{i}\n' + '-' * 20)
    model = WordEmbAvg(INPUT_DIM, embed_size, hidden_size,
                OUTPUT_SIZE, PADDING_IDX, dropout_p=dropout)
    
    tm = TrainingModule(model, lr, POS_WEIGHT, USE_CUDA, EPOCHS)
    
    best_models = tm.train_model(train_it, val_it)
    
    for metric, best_model in best_models.items():
        row = [i, embed_size, hidden_size, dropout,
               lr, EPOCHS, best_model.accuracy,
               best_model.precision, best_model.recall, metric]
        df.loc[len(df)] = row
        if best_acc[0] is None or best_model.accuracy > best_acc[1]:
            best_acc = (copy.deepcopy(best_model.model), best_model.accuracy)
        if best_rec[0] is None or best_model.recall > best_rec[1]:
            best_rec = (copy.deepcopy(best_model.model), best_model.recall)
        if best_prec[0] is None or isnan(best_prec[1]) or\
          best_model.precision > best_prec[1]:
            best_prec = (copy.deepcopy(best_model.model), best_model.precision)
    
    print('-' * 20 + '\n')


Architecture #0
--------------------
Epoch 0: Dev Accuracy: 0.2500; Dev Precision: 0.0000; Dev Recall: 0.0000; Dev Loss:0.9438
Epoch 1: Dev Accuracy: 0.2500; Dev Precision: 0.0000; Dev Recall: 0.0000; Dev Loss:0.9252
Epoch 2: Dev Accuracy: 0.2500; Dev Precision: 0.0000; Dev Recall: 0.0000; Dev Loss:0.9067
Epoch 3: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.8924
Epoch 4: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.8809
Epoch 5: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.8704
Epoch 6: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.8395
Epoch 7: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.8410
Epoch 8: Dev Accuracy: 0.5000; Dev Precision: 0.0000; Dev Recall: 0.0000; Dev Loss:0.8757
Epoch 9: Dev Accuracy: 0.5000; Dev Precision: 0.0000; Dev Recall: 0.0000; Dev Loss:0.9089
--------------------

Architecture #1
--------------------
Epoch 0: Dev Accura

Epoch 8: Dev Accuracy: 0.2500; Dev Precision: 0.2500; Dev Recall: 1.0000; Dev Loss:0.9183
Epoch 9: Dev Accuracy: 0.2500; Dev Precision: 0.2500; Dev Recall: 1.0000; Dev Loss:0.9179
--------------------

Architecture #9
--------------------
Epoch 0: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.8777
Epoch 1: Dev Accuracy: 0.2500; Dev Precision: 0.2500; Dev Recall: 1.0000; Dev Loss:0.8961
Epoch 2: Dev Accuracy: 0.5000; Dev Precision: 0.3333; Dev Recall: 1.0000; Dev Loss:0.8882
Epoch 3: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.8478
Epoch 4: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.8140
Epoch 5: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.7716
Epoch 6: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.7232
Epoch 7: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.7064
Epoch 8: Dev Accuracy: 0.7500; Dev Precision

Epoch 4: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.8935
Epoch 5: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.8930
Epoch 6: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.8925
Epoch 7: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.8920
Epoch 8: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.8914
Epoch 9: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.8908
--------------------

Architecture #18
--------------------
Epoch 0: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.8755
Epoch 1: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.8638
Epoch 2: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.8324
Epoch 3: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.7923
Epoch 4: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev 

Epoch 0: Dev Accuracy: 0.2500; Dev Precision: 0.2500; Dev Recall: 1.0000; Dev Loss:0.9042
Epoch 1: Dev Accuracy: 0.2500; Dev Precision: 0.2500; Dev Recall: 1.0000; Dev Loss:0.9041
Epoch 2: Dev Accuracy: 0.2500; Dev Precision: 0.2500; Dev Recall: 1.0000; Dev Loss:0.9042
Epoch 3: Dev Accuracy: 0.2500; Dev Precision: 0.2500; Dev Recall: 1.0000; Dev Loss:0.9041
Epoch 4: Dev Accuracy: 0.2500; Dev Precision: 0.2500; Dev Recall: 1.0000; Dev Loss:0.9041
Epoch 5: Dev Accuracy: 0.2500; Dev Precision: 0.2500; Dev Recall: 1.0000; Dev Loss:0.9041
Epoch 6: Dev Accuracy: 0.2500; Dev Precision: 0.2500; Dev Recall: 1.0000; Dev Loss:0.9040
Epoch 7: Dev Accuracy: 0.2500; Dev Precision: 0.2500; Dev Recall: 1.0000; Dev Loss:0.9041
Epoch 8: Dev Accuracy: 0.2500; Dev Precision: 0.2500; Dev Recall: 1.0000; Dev Loss:0.9042
Epoch 9: Dev Accuracy: 0.2500; Dev Precision: 0.2500; Dev Recall: 1.0000; Dev Loss:0.9043
--------------------

Architecture #27
--------------------
Epoch 0: Dev Accuracy: 0.7500; Dev Preci

Epoch 7: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.8890
Epoch 8: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.8880
Epoch 9: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.8874
--------------------

Architecture #35
--------------------
Epoch 0: Dev Accuracy: 0.2500; Dev Precision: 0.2500; Dev Recall: 1.0000; Dev Loss:0.9117
Epoch 1: Dev Accuracy: 0.2500; Dev Precision: 0.2500; Dev Recall: 1.0000; Dev Loss:0.9115
Epoch 2: Dev Accuracy: 0.2500; Dev Precision: 0.2500; Dev Recall: 1.0000; Dev Loss:0.9114
Epoch 3: Dev Accuracy: 0.2500; Dev Precision: 0.2500; Dev Recall: 1.0000; Dev Loss:0.9113
Epoch 4: Dev Accuracy: 0.2500; Dev Precision: 0.2500; Dev Recall: 1.0000; Dev Loss:0.9112
Epoch 5: Dev Accuracy: 0.2500; Dev Precision: 0.2500; Dev Recall: 1.0000; Dev Loss:0.9112
Epoch 6: Dev Accuracy: 0.2500; Dev Precision: 0.2500; Dev Recall: 1.0000; Dev Loss:0.9110
Epoch 7: Dev Accuracy: 0.2500; Dev Precisio

...

Epoch 4: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.8596
Epoch 5: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.8489
Epoch 6: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.8442
Epoch 7: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.8279
Epoch 8: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.8175
Epoch 9: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.8107
--------------------

Architecture #200
--------------------
Epoch 0: Dev Accuracy: 0.2500; Dev Precision: 0.2500; Dev Recall: 1.0000; Dev Loss:0.9022
Epoch 1: Dev Accuracy: 0.2500; Dev Precision: 0.2500; Dev Recall: 1.0000; Dev Loss:0.9027
Epoch 2: Dev Accuracy: 0.2500; Dev Precision: 0.2500; Dev Recall: 1.0000; Dev Loss:0.9025
Epoch 3: Dev Accuracy: 0.2500; Dev Precision: 0.2500; Dev Recall: 1.0000; Dev Loss:0.9024
Epoch 4: Dev Accuracy: 0.2500; Dev Prec

Epoch 0: Dev Accuracy: 0.2500; Dev Precision: 0.2500; Dev Recall: 1.0000; Dev Loss:0.9212
Epoch 1: Dev Accuracy: 0.2500; Dev Precision: 0.2500; Dev Recall: 1.0000; Dev Loss:0.9139
Epoch 2: Dev Accuracy: 0.2500; Dev Precision: 0.2500; Dev Recall: 1.0000; Dev Loss:0.9097
Epoch 3: Dev Accuracy: 0.2500; Dev Precision: 0.2500; Dev Recall: 1.0000; Dev Loss:0.9055
Epoch 4: Dev Accuracy: 0.2500; Dev Precision: 0.2500; Dev Recall: 1.0000; Dev Loss:0.9022
Epoch 5: Dev Accuracy: 0.2500; Dev Precision: 0.0000; Dev Recall: 0.0000; Dev Loss:0.9014
Epoch 6: Dev Accuracy: 0.5000; Dev Precision: 0.3333; Dev Recall: 1.0000; Dev Loss:0.8957
Epoch 7: Dev Accuracy: 0.5000; Dev Precision: 0.3333; Dev Recall: 1.0000; Dev Loss:0.8935
Epoch 8: Dev Accuracy: 0.5000; Dev Precision: 0.3333; Dev Recall: 1.0000; Dev Loss:0.8895
Epoch 9: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.8815
--------------------

Architecture #209
--------------------
Epoch 0: Dev Accuracy: 0.2500; Dev Prec

Epoch 7: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.6196
Epoch 8: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.6519
Epoch 9: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.9152
--------------------

Architecture #217
--------------------
Epoch 0: Dev Accuracy: 0.5000; Dev Precision: 0.3333; Dev Recall: 1.0000; Dev Loss:0.8900
Epoch 1: Dev Accuracy: 0.5000; Dev Precision: 0.3333; Dev Recall: 1.0000; Dev Loss:0.8878
Epoch 2: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.8798
Epoch 3: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.8741
Epoch 4: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.8674
Epoch 5: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.8590
Epoch 6: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.8533
Epoch 7: Dev Accuracy: 0.7500; Dev Precision: 0.

Epoch 3: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.5960
Epoch 4: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.5209
Epoch 5: Dev Accuracy: 1.0000; Dev Precision: 1.0000; Dev Recall: 1.0000; Dev Loss:0.4444
Epoch 6: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.6293
Epoch 7: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.5162
Epoch 8: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:2.0884
Epoch 9: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.4384
--------------------

Architecture #226
--------------------
Epoch 0: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.8778
Epoch 1: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.8754
Epoch 2: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.8715
Epoch 3: Dev Accuracy: 0.7500; Dev Precision:

Epoch 0: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:1.0457
Epoch 1: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.7902
Epoch 2: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.7737
Epoch 3: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.7820
Epoch 4: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.7531
Epoch 5: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.6696
Epoch 6: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.6227
Epoch 7: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.7089
Epoch 8: Dev Accuracy: 0.5000; Dev Precision: 0.0000; Dev Recall: 0.0000; Dev Loss:0.8102
Epoch 9: Dev Accuracy: 0.5000; Dev Precision: 0.0000; Dev Recall: 0.0000; Dev Loss:0.8863
--------------------

Architecture #235
--------------------
Epoch 0: Dev Accuracy: 0.2500; Dev Precision:

## Save results of model training

In [14]:
df

,architecture,embeddings,hidden,dropouts,learning_rate,epochs,dev_acc,dev_prec,dev_recall,metric
0,0,32,10,0,0.0100,10,0.75,NaN,0.0,accuracy
1,0,32,10,0,0.0100,10,0.25,0.0,0.0,precision
2,0,32,10,0,0.0100,10,0.25,0.0,0.0,recall
3,1,32,10,0,0.0010,10,0.75,NaN,0.0,accuracy
4,1,32,10,0,0.0010,10,0.75,NaN,0.0,precision
...,...,...,...,...,...,...,...,...,...,...
715,238,256,50,0.75,0.0010,10,0.75,0.5,1.0,precision
716,238,256,50,0.75,0.0010,10,0.75,0.5,1.0,recall
717,239,256,50,0.75,0.0001,10,0.25,0.0,0.0,accuracy
718,239,256,50,0.75,0.0001,10,0.25,0.0,0.0,precision


In [15]:
SAVE_PREFIX = '../results/SimpleNNCustom_'
df.to_csv(f'{SAVE_PREFIX}models.csv')
torch.save(best_acc[0], f'{SAVE_PREFIX}best_acc.pt')
torch.save(best_rec[0], f'{SAVE_PREFIX}best_rec.pt')
torch.save(best_prec[0], f'{SAVE_PREFIX}best_prec.pt')

/Users/compileherlaptop/opt/anaconda3/lib/python3.7/site-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type WordEmbAvg. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
